# Load synthetic DNA dataset
## Convert sequences to uppercase strings (just in case)
### Store in list: `seqs`

In [1]:
#load dataset 
import pandas as pd
import numpy as np
from itertools import product

#Store the dataset
df = pd.read_csv("synthetic_dna_dataset.csv")
seqs = df["Sequence"].astype(str).str.upper().tolist()


# Function: longest_repeat_with_base(sequence)
### Returns: (max_run_length, repeated_nucleotide)
## Example: "AAACGT" → (3, 'A')

In [2]:
def longest_repeat_with_base(sequence):
    # Returns:
    # length of the longest contiguous repeat
    # nucleotide responsible for that repeat
    max_run = 1
    current_run = 1
    max_base = sequence[0]

    for i in range(1, len(sequence)):
        if sequence[i] == sequence[i - 1]:
            current_run += 1
            if current_run > max_run:
                max_run = current_run
                max_base = sequence[i]
        else:
            current_run = 1

    return max_run, max_base

#The algorithm is mentioned in the report draft

# Apply function to all sequences
## Add two new columns to dataframe:
### - Longest_Repeat_Length
### - Longest_Repeat_Base

In [3]:
df[["Longest_Repeat_Length", "Longest_Repeat_Base"]] = (
    df["Sequence"]
    .apply(longest_repeat_with_base)
    .apply(pd.Series)
)

df[["Sequence", "Longest_Repeat_Length", "Longest_Repeat_Base"]]


,Sequence,Longest_Repeat_Length,Longest_Repeat_Base
0,CTTTCGGGATACTTTTGGGATGGTCTTGGTCAAGGGTTTTAGCCCG...,4,T
1,TTGACCAAATTTGATTGGAAGTGGTAAGCGCGTATTCCTAGCATCA...,5,T
2,GCGTGAGTTCTAATTTAAAAAGTCGTAACACGTACCCCGGCGTGTA...,5,A
3,ACTACGCGGACAAGAACCAACAGAACCTGGTTTTCGCAAGGGAGTG...,4,T
4,TTCAATGCAGATTGAAAGTTACTTTCATCTGCCCTATGGGTCCCTT...,3,A
...,...,...,...
2995,GATCAGCCCATACACCAAATCAATTGCATACATGTCCGATGTAACA...,3,C
2996,TGTTGTGTGTCTGATGATAGGTCATACCGCCTCGAAACATCACCAT...,4,A
2997,GACCCACTAAAAGTCTTCGTCTCCTTCCGATGGGAATTTTCGCCGA...,4,A
2998,CCAAAGGATATCTGTAATTGTTGCAGCGCCCCTACAATTTGAGCAC...,4,C


First 5 rows show repeats of length 3-5, mostly T and A repeats.

# Function: shannon_entropy_base(seq)
## Calculates randomness of A,C,G,T distribution
## Higher entropy = more balanced nucleotide distribution

In [4]:
import numpy as np

def shannon_entropy_base(seq: str) -> float:
    seq = str(seq).upper()
    counts = np.array([
        seq.count("A"),
        seq.count("C"),
        seq.count("G"),
        seq.count("T")
    ], dtype=float)

    total = counts.sum()
    if total == 0:
        return 0.0

    p = counts / total
    p = p[p > 0]  # avoid log(0)
    return float(-(p * np.log2(p)).sum())

df["Shannon_Entropy_Base"] = df["Sequence"].apply(shannon_entropy_base)
df[["Sequence", "Shannon_Entropy_Base"]].head()


,Sequence,Shannon_Entropy_Base
0,CTTTCGGGATACTTTTGGGATGGTCTTGGTCAAGGGTTTTAGCCCG...,1.973815
1,TTGACCAAATTTGATTGGAAGTGGTAAGCGCGTATTCCTAGCATCA...,1.992483
2,GCGTGAGTTCTAATTTAAAAAGTCGTAACACGTACCCCGGCGTGTA...,1.986869
3,ACTACGCGGACAAGAACCAACAGAACCTGGTTTTCGCAAGGGAGTG...,1.962509
4,TTCAATGCAGATTGAAAGTTACTTTCATCTGCCCTATGGGTCCCTT...,1.988047


# Describe the entropy values
### Mean: ~1.978 (max possible: 2.0)
### Range: 1.847 - 2.000

In [5]:
df["Shannon_Entropy_Base"].describe()


count    3000.000000
mean        1.978182
std         0.018361
min         1.846713
25%         1.970229
50%         1.983165
75%         1.991433
max         2.000000
Name: Shannon_Entropy_Base, dtype: float64

Sequences are nearly perfectly balanced (close to maximum entropy 2.0).

# Create all possible 3-mers: AAA, AAC, AAG, ..., TTT
### Total: 64 possible combinations (4^3)
## Create index for fast lookup

In [6]:
#Build the 3-mer vocabulary (fixed order, size = 64)
K = 3
ALPHABET = ["A", "C", "G", "T"]

kmer_list = [''.join(p) for p in product(ALPHABET, repeat=K)]
kmer_index = {kmer: i for i, kmer in enumerate(kmer_list)}


Prepare for 3-mer frequency analysis.

# Function: compute_kmer3_vector(sequence)
## Count each 3-mer in sequence
## Normalize by total number of 3-mers (98 for 100-length seq)

In [7]:
#EXACT k-mer extraction + normalization (PDF-correct)
def compute_kmer3_vector(sequence):
    vector = np.zeros(len(kmer_list), dtype=float)
    total_kmers = len(sequence) - K + 1  # = 98 for length 100

    for i in range(total_kmers):
        kmer = sequence[i:i+K]
        if kmer in kmer_index:   # assumes only A,C,G,T
            vector[kmer_index[kmer]] += 1

    # normalize exactly as in PDF
    vector /= total_kmers
    return vector


Apply to all 3000 sequences
Shape: (3000, 64) → 3000 sequences × 64 features\
Each sequence represented as 64-dimensional vector.

In [8]:
#Apply to entire dataset
X_kmer3 = np.vstack([compute_kmer3_vector(seq) for seq in seqs])

print(X_kmer3.shape)  # (N, 64)


(3000, 64)


# Create readable DataFrame with columns like k3_AAA, k3_AAC, etc.
## First row sum = 1.0 (verifies normalization)

In [9]:
#(Optional but recommended) Put into DataFrame
kmer_columns = [f"k3_{kmer}" for kmer in kmer_list]
df_kmer3 = pd.DataFrame(X_kmer3, columns=kmer_columns)

df_kmer3.head()


,k3_AAA,k3_AAC,k3_AAG,k3_AAT,k3_ACA,k3_ACC,k3_ACG,k3_ACT,k3_AGA,k3_AGC,...,k3_TCG,k3_TCT,k3_TGA,k3_TGC,k3_TGG,k3_TGT,k3_TTA,k3_TTC,k3_TTG,k3_TTT
0,0.020408,0.020408,0.020408,0.010204,0.010204,0.010204,0.010204,0.020408,0.030612,0.010204,...,0.010204,0.010204,0.000000,0.020408,0.040816,0.000000,0.030612,0.010204,0.051020,0.061224
1,0.020408,0.020408,0.020408,0.020408,0.020408,0.010204,0.020408,0.010204,0.010204,0.030612,...,0.000000,0.010204,0.020408,0.010204,0.020408,0.010204,0.010204,0.010204,0.051020,0.040816
2,0.030612,0.030612,0.020408,0.010204,0.020408,0.040816,0.010204,0.020408,0.020408,0.000000,...,0.020408,0.020408,0.010204,0.000000,0.010204,0.020408,0.020408,0.020408,0.010204,0.010204
3,0.000000,0.040816,0.030612,0.000000,0.020408,0.030612,0.020408,0.010204,0.030612,0.000000,...,0.010204,0.000000,0.020408,0.000000,0.030612,0.000000,0.010204,0.010204,0.010204,0.030612
4,0.010204,0.010204,0.010204,0.020408,0.010204,0.010204,0.000000,0.020408,0.010204,0.020408,...,0.010204,0.010204,0.020408,0.030612,0.020408,0.000000,0.010204,0.030612,0.020408,0.020408


In [10]:
df_kmer3.iloc[0].sum()


np.float64(1.0)

# Create all possible 4-mers
### Total: 256 combinations (4^4)

In [11]:
#Build 4-mer vocabulary (size = 256)
K = 4
ALPHABET = ["A", "C", "G", "T"]

kmer_list_4 = [''.join(p) for p in product(ALPHABET, repeat=K)]
kmer_index_4 = {kmer: i for i, kmer in enumerate(kmer_list_4)}

len(kmer_list_4)  # should be 256


256

# Function: compute_kmer4_vector(sequence)
## Same logic as 3-mer but with k=4
## Normalized by total 4-mers (97 for 100-length seq)

In [12]:
#Compute normalized 4-mer vector per sequence
def compute_kmer4_vector(sequence: str) -> np.ndarray:
    vector = np.zeros(len(kmer_list_4), dtype=float)
    total_kmers = len(sequence) - K + 1  # for length 100 -> 97

    for i in range(total_kmers):
        kmer = sequence[i:i+K]
        if kmer in kmer_index_4:      # assumes only A,C,G,T
            vector[kmer_index_4[kmer]] += 1

    vector /= total_kmers  # normalization exactly like PDF
    return vector


In [13]:
#Apply to all sequences
X_kmer4 = np.vstack([compute_kmer4_vector(seq) for seq in seqs])

print("X_kmer4 shape:", X_kmer4.shape)  # (N, 256)
print("First row sum:", X_kmer4[0].sum())  # should be ~1.0


X_kmer4 shape: (3000, 256)
First row sum: 1.0


In [14]:
#Optional: turn into a DataFrame (easy to save/inspect)
k4_cols = [f"k4_{kmer}" for kmer in kmer_list_4]
df_kmer4 = pd.DataFrame(X_kmer4, columns=k4_cols)

df_kmer4.head()


,k4_AAAA,k4_AAAC,k4_AAAG,k4_AAAT,k4_AACA,k4_AACC,k4_AACG,k4_AACT,k4_AAGA,k4_AAGC,...,k4_TTCG,k4_TTCT,k4_TTGA,k4_TTGC,k4_TTGG,k4_TTGT,k4_TTTA,k4_TTTC,k4_TTTG,k4_TTTT
0,0.010309,0.010309,0.000000,0.000000,0.000000,0.010309,0.010309,0.000000,0.000000,0.000000,...,0.010309,0.000000,0.000000,0.020619,0.030928,0.000000,0.020619,0.010309,0.010309,0.020619
1,0.000000,0.010309,0.000000,0.010309,0.010309,0.000000,0.000000,0.010309,0.000000,0.010309,...,0.000000,0.000000,0.020619,0.010309,0.010309,0.010309,0.010309,0.000000,0.010309,0.020619
2,0.020619,0.000000,0.010309,0.000000,0.020619,0.010309,0.000000,0.000000,0.010309,0.000000,...,0.000000,0.020619,0.000000,0.000000,0.010309,0.000000,0.010309,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.010309,0.020619,0.010309,0.000000,0.010309,0.000000,...,0.010309,0.000000,0.000000,0.000000,0.010309,0.000000,0.010309,0.010309,0.000000,0.010309
4,0.000000,0.000000,0.010309,0.000000,0.010309,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.010309,0.010309,0.000000,0.000000,0.000000,0.010309,0.010309,0.000000


In [15]:
#Optional: save a new CSV with your computed k=4 features
#out4 = pd.concat([df.reset_index(drop=True), df_kmer4.reset_index(drop=True)], axis=1)
#out4_path = "/mnt/data/dataset_with_my_kmer4.csv"
#out4.to_csv(out4_path, index=False)
#out4_path


# Create 6-mer features (4096 dimensions!)
### Apply same normalization logic

In [16]:
# ===== k=6 features =====
K = 6
ALPHABET = ["A", "C", "G", "T"]

kmer_list_6 = [''.join(p) for p in product(ALPHABET, repeat=K)]
kmer_index_6 = {kmer: i for i, kmer in enumerate(kmer_list_6)}

def compute_kmer6_vector(sequence: str) -> np.ndarray:
    vector = np.zeros(len(kmer_list_6), dtype=float)
    total_kmers = len(sequence) - K + 1  # for length 100 -> 95

    for i in range(total_kmers):
        kmer = sequence[i:i+K]
        if kmer in kmer_index_6:
            vector[kmer_index_6[kmer]] += 1

    vector /= total_kmers  # normalization like PDF
    return vector

X_kmer6 = np.vstack([compute_kmer6_vector(seq) for seq in seqs])

print("X_kmer6 shape:", X_kmer6.shape)   # (N, 4096)
print("First row sum:", X_kmer6[0].sum())  # ~1.0


X_kmer6 shape: (3000, 4096)
First row sum: 1.0


# Compare k=3, k=4, k=6 using Linear SVM
## Results:
### - k=3: Accuracy 0.2417, F1 0.2415
### - k=4: Accuracy 0.2367, F1 0.2368
### - k=6: Accuracy 0.2233, F1 0.2202

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score

# Pick ONE target
y = df["Class_Label"].values
# y = df["Disease_Risk"].values

idx = np.arange(len(y))

train_idx, test_idx = train_test_split(
    idx, test_size=0.2, random_state=42, stratify=y
)

y_train, y_test = y[train_idx], y[test_idx]

def train_eval_linear_svm(X, train_idx, test_idx, y_train, y_test):
    X_train, X_test = X[train_idx], X[test_idx]

    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train)
    X_test_s  = scaler.transform(X_test)

    model = SVC(kernel="rbf", C=1.0)
    model.fit(X_train_s, y_train)
    pred = model.predict(X_test_s)

    acc = accuracy_score(y_test, pred)
    f1  = f1_score(y_test, pred, average="macro")
    return acc, f1

acc3, f13 = train_eval_linear_svm(X_kmer3, train_idx, test_idx, y_train, y_test)
acc4, f14 = train_eval_linear_svm(X_kmer4, train_idx, test_idx, y_train, y_test)
acc6, f16 = train_eval_linear_svm(X_kmer6, train_idx, test_idx, y_train, y_test)

print("=== Comparison (Linear SVM, same split) ===")
print(f"k=3 → Accuracy: {acc3:.4f} | Macro-F1: {f13:.4f}")
print(f"k=4 → Accuracy: {acc4:.4f} | Macro-F1: {f14:.4f}")
print(f"k=6 → Accuracy: {acc6:.4f} | Macro-F1: {f16:.4f}")


=== Comparison (Linear SVM, same split) ===
k=3 → Accuracy: 0.2183 | Macro-F1: 0.2179
k=4 → Accuracy: 0.2517 | Macro-F1: 0.2510
k=6 → Accuracy: 0.2383 | Macro-F1: 0.2349


Classification performance decreases as k increases, with k=3 outperforming k=4 and k=6. This suggests that shorter k-mers provide more robust and generalizable representations for the given sequence length and dataset size, while higher k introduces sparsity and overfitting
The dataset simply doesn’t support k=6.


In [18]:
# def longest_repeat_with_base(sequence):
#     """
#     Returns:
#     - length of the longest contiguous repeat
#     - nucleotide responsible for that repeat
#     """
#     max_run = 1
#     current_run = 1
#     max_base = sequence[0]

#     for i in range(1, len(sequence)):
#         if sequence[i] == sequence[i - 1]:
#             current_run += 1
#             if current_run > max_run:
#                 max_run = current_run
#                 max_base = sequence[i]
#         else:
#             current_run = 1

#     return max_run, max_base


In [21]:
# df[["Longest_Repeat_Length", "Longest_Repeat_Base"]] = (
#     df["Sequence"]
#     .apply(longest_repeat_with_base)
#     .apply(pd.Series)
# )

# df[["Sequence", "Longest_Repeat_Length", "Longest_Repeat_Base"]].head()


In [22]:
# import numpy as np

# def shannon_entropy_base(seq: str) -> float:
#     seq = str(seq).upper()
#     counts = np.array([
#         seq.count("A"),
#         seq.count("C"),
#         seq.count("G"),
#         seq.count("T")
#     ], dtype=float)

#     total = counts.sum()
#     if total == 0:
#         return 0.0

#     p = counts / total
#     p = p[p > 0]  # avoid log(0)
#     return float(-(p * np.log2(p)).sum())

# df["Shannon_Entropy_Base"] = df["Sequence"].apply(shannon_entropy_base)
# df[["Sequence", "Shannon_Entropy_Base"]].head()


In [23]:
# df["Shannon_Entropy_Base"].describe()


# Calculate entropy of 3-mer distributions
## Compare with single-base entropy:
### Base entropy: ~1.98 (near max)
### 3-mer entropy: ~5.26-5.65 (high but not max)

In [24]:
def shannon_entropy_from_probs(p_vec):
    p = p_vec[p_vec > 0]
    return float(-(p * np.log2(p)).sum())

df["Shannon_Entropy_kmer3"] = np.array([shannon_entropy_from_probs(v) for v in X_kmer3])
df[["Shannon_Entropy_Base", "Shannon_Entropy_kmer3"]]


,Shannon_Entropy_Base,Shannon_Entropy_kmer3
0,1.973815,5.398807
1,1.992483,5.649106
2,1.986869,5.526657
3,1.962509,5.439623
4,1.988047,5.593903
...,...,...
2995,1.980472,5.528959
2996,1.996023,5.516253
2997,1.986869,5.417177
2998,1.992332,5.645123


Interpretation

These values are very close to 2

That means:

A, C, G, T occur in nearly equal proportions

The sequences are compositionally balanced

There is no strong single-base dominance

 In plain English:

Most sequences look “random” at the single-base level.

second column Interpretation

Values are high, but not maximal

This means:

Many different 3-mers appear

But not all 3-mers are equally frequent

Some motifs occur more often than others

📌 In plain English:

The sequences are diverse, but still contain local patterns and repeats.